<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/task-amenability/task_amenability_species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym==0.17.3
!pip install stable-baselines==2.10.2
!pip install h5py==3.0.0
!pip install tensorflow==1.13.1
!pip install tensorboard==1.13.1
!pip install Keras==2.2.4
!pip install Keras-Applications==1.0.8
!pip install Keras-Preprocessing==1.1.2
!pip install keras-rl==0.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 27.7 MB/s 
     |████████████████████████████████| 1.0 MB 56.1 MB/s 
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654653 sha256=0ce53b9e96b0993a078004a19d30dc30f316fb977d1191254114a36b0d670278
  Stored in directory: /root/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 32.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 37.7 MB/s 
  Attempting uninstall: h5py
    Found existing ins

In [ ]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

# get downstream task model
shutil.copy(PARENT_DIR + '/task-amenability/species/model.pt', 'species_model.pt')

shutil.copy(PARENT_DIR + '/task-amenability/object/model.pt', 'object_model.pt')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'object_model.pt'

In [ ]:
# clone the repository and move data file
!git clone https://github.com/YolandaMDavis/wildtrack-iqa.git
!mv WildTrack_Raw.zip wildtrack-iqa/.
!mv species_model.pt wildtrack-iqa/.
!mv object_model.pt wildtrack-iqa/.
%cd "wildtrack-iqa"
!git checkout task-amenability
!mv species_model.pt taskamenability/taskpredictor/species/model.pt
!mv object_model.pt taskamenability/taskpredictor/objdet/model.pt


Cloning into 'wildtrack-iqa'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 99 (delta 36), reused 71 (delta 17), pack-reused 0
Unpacking objects: 100% (99/99), done.
/content/wildtrack-iqa
Branch 'task-amenability' set up to track remote branch 'task-amenability' from 'origin'.
Switched to a new branch 'task-amenability'


In [ ]:
from zipfile import ZipFile
# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
%cd "wildtrack-iqa"

import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
import albumentations as A

with open('taskamenability/taskpredictor/species/class_mapping.json') as data:
    mappings = json.load(data)

class_mapping = {item['model_idx']: item['class_name'] for item in mappings}
device = torch.device('cpu')
model = torch.jit.load('taskamenability/taskpredictor/species/model.pt').to(device)
transforms = A.load('taskamenability/taskpredictor/species/transforms.json')

/content/wildtrack-iqa


In [ ]:
from torch.utils import data
import csv
import random
import torch
import yaml
import cv2
import os

from time import process_time
from torch.utils.data import Dataset

root_dir = '/content/wildtrack-iqa'
data_dir = root_dir + '/RAW'

def get_key(val, item_dict):
    for key, value in item_dict.items():
        if val == value:
            return key
    return -1

def createImageSamples(img_shape, num_samples):
  x = np.random.rand(num_samples, img_shape[0], img_shape[1], img_shape[2])
  y = np.random.randint(low=0, high=2, size=(num_samples, 1))
  return (x,y)

def generate_data_sets(sample_percentage=1):

    image_reference_list = []
    subdirectories = list(os.walk(data_dir, topdown=False))[:-1]

    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        subjective_score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]
        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        for image in images:
            class_name = species if animal_class == 'Unknown' else species + ': ' + animal_class
            class_index = get_key(class_name, class_mapping)
            if class_index > -1:
              image_reference = (image_location, species, animal_class, image, class_index, subjective_score)
              image_reference_list.append(image_reference)
    
    # add percentage of generated data
    #samples = int(len(image_reference_list) * .20)
    #for i in range(0,samples):
    #  image_reference_list.append(('sample','sample','sample','sample',0))

    # shuffle then split
    seed = 1234
    random.Random(seed).shuffle(image_reference_list)

    train_index = int(len(image_reference_list) * 0.6 * sample_percentage)
    test_index = -int(len(image_reference_list) * 0.2 * sample_percentage) 
    val_index = test_index * 2

    training = image_reference_list[:train_index]
    validation = image_reference_list[val_index:test_index]
    testing = image_reference_list[test_index:]
    return training, validation, testing

def convert_image(img_test):
    image = Image.open(img_test)
    #image = image.convert('L')
    #image = image.resize((1024, 768))
    image = np.array(image)
    #image = np.expand_dims(image, axis=2)
    image = transforms(image=image)['image']
    x = torch.from_numpy(image).to(device)
    return x


def transform_images(images):
  x = []
  y = []
  for img in images:
      image_name = img[0]+'/'+img[3]
      image = convert_image(image_name)
      x.append(image)
      y.append(img[4])

  return np.array(x), np.array(y)


In [ ]:
# take a percentage of the full data set as a training/test/validation sample
sample_size=.1
train, valid, test = generate_data_sets(sample_size)
x_train, y_train = transform_images(train)
x_val, y_val = transform_images(valid)
x_test, y_test = transform_images(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


**Execute the below installation only in Colab environment**

In [ ]:
from taskamenability.interface import PPOInterface

img_shape = (512,512,3)
task_predictor = model
interface = PPOInterface(x_train, y_train, x_val, y_val, x_test, y_test, task_predictor, img_shape)

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [ ]:
interface.train(6)

Training started for 6 episodes:
[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Reward: 0.05208329571339876
--------------------------------------
| approxkl           | 0.006068525   |
| clipfrac           | 0.07211539    |
| explained_variance | -19.3         |
| fps                | 3             |
| n_updates          | 1             |
| policy_entropy     | 1.4189036     |
| policy_loss        | -0.0009359652 |
| serial_timesteps   | 104           |
| time_elapsed       | 2.24e-05      |
| total_timesteps    | 104           |
| value_loss         | 25669.

In [ ]:
preds = []
class_preds = []
for item in interface.x_holdout:
  obs = item.permute(2, 0, 1).unsqueeze(dim=0).float()
  y_pred = model(obs)
  y_pred = y_pred.argmax(dim=1).squeeze().item()
  class_preds.append(y_pred)
  preds.append(interface.model.predict(item))

In [ ]:
for i in range(len(test)):
  print(test[i])
  print(preds[i])
  print(class_preds[i])
  print('\n')

('/content/wildtrack-iqa/RAW/Rhino_Black_5', 'Rhino', 'Black', 'c048cfb4ccff11eabc8b0242ac1c0002.jpg', 0, 5)
(array([1.], dtype=float32), None)
0


('/content/wildtrack-iqa/RAW/Lion_African_5', 'Lion', 'African', '4dc339ec0db011eb9d950242ac110002.jpg', 12, 5)
(array([1.], dtype=float32), None)
12


('/content/wildtrack-iqa/RAW/Otter_Eurasian_2', 'Otter', 'Eurasian', '11cf15f0d77811eab34b0242ac110002.jpg', 2, 2)
(array([0.], dtype=float32), None)
14


('/content/wildtrack-iqa/RAW/Otter_Eurasian_5', 'Otter', 'Eurasian', 'ae21bcde373411eb95a500155d4144f4.jpg', 2, 5)
(array([1.], dtype=float32), None)
4


('/content/wildtrack-iqa/RAW/Cheetah_Southeast_African_5', 'Cheetah', 'Southeast African', '73e7b314926c11eb983000155d759f45.jpg', 11, 5)
(array([0.], dtype=float32), None)
4


('/content/wildtrack-iqa/RAW/Leopard_African_4', 'Leopard', 'African', 'aaccf7cad63011eab2300242ac1c0002.jpg', 7, 4)
(array([0.90999657], dtype=float32), None)
0


('/content/wildtrack-iqa/RAW/Otter_Eurasian_3', 'O

In [ ]:
# how to extract quality mode from interface and use for prediction
# eventually this should be a pytorch model and we'll need to account

preds

[(array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 (array([1.], dtype=float32), None),
 